In [8]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

train_df= pd.read_csv("train.csv")
test_data= pd.read_csv("test.csv")
#print(train_df.isna().sum())
train_df = train_df.drop(columns = ["Cabin", "Name"])
test_df = test_data.drop(columns = ["Cabin", "Name"])

X_cat = train_df[['HomePlanet', 'CryoSleep', 'Destination', 'VIP']]
X_num = train_df[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck']]
y =  train_df['Transported']
Z_cat = test_df[['HomePlanet', 'CryoSleep', 'Destination', 'VIP']]
Z_num = test_df[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck']]

imp_cat = SimpleImputer(strategy = "most_frequent")
X_cat = imp_cat.fit_transform(X_cat)
Z_cat = imp_cat.transform(Z_cat)

imp_num = SimpleImputer()
X_num = imp_num.fit_transform(X_num)
Z_num = imp_num.transform(Z_num)
X_train = np.append(X_cat, X_num, axis=1)
X_test = np.append(Z_cat, Z_num, axis=1)
X_train = pd.DataFrame(X_train, columns = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP',
                                           'Age', 'RoomService', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck'])
X_test = pd.DataFrame(X_test, columns = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP',
                                           'Age', 'RoomService', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck'])



X_train_dummies = pd.get_dummies(X_train[['HomePlanet', 'Destination']], drop_first = True)
X_test_dummies = pd.get_dummies(X_test[['HomePlanet', 'Destination']], drop_first = True)

X_train = pd.concat([X_train,X_train_dummies], axis = 1)
X_test = pd.concat([X_test,X_test_dummies], axis = 1)
#print(X_test.shape)
X_train = X_train.drop(columns = ['HomePlanet', 'Destination'], axis = 1)
X_test = X_test.drop(columns = ['HomePlanet', 'Destination'], axis = 1)
print(X_test.isna().sum())
#print(X_train.shape)
sgbt = GradientBoostingClassifier()
params_sgbt = {"n_estimators": np.arange(0,301,1),
             "max_depth": np.arange(0,11,1),
             "subsample": np.arange(0.1,1.0,0.1),
             "max_features": np.arange(0.1,1.0,0.1),
             }
sgbt_randomized = RandomizedSearchCV(sgbt, param_distributions= params_sgbt, cv = 10, random_state= 1)
sgbt_randomized.fit(X_train,y)
hyperparams = sgbt_randomized.best_params_
sgbt = GradientBoostingClassifier(n_estimators= hyperparams["n_estimators"], max_depth=hyperparams["max_depth"],
                                  subsample=hyperparams["subsample"],max_features=hyperparams["max_features"])
sgbt.fit(X_train,y)
Predictions = sgbt.predict(X_test)
print(Predictions.shape)
Submission = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': Predictions})
Submission.to_csv("Submission.csv", index = False)
print("Mission Successful")

CryoSleep                    0
VIP                          0
Age                          0
RoomService                  0
FoodCourt                    0
ShoppingMall                 0
Spa                          0
VRDeck                       0
HomePlanet_Europa            0
HomePlanet_Mars              0
Destination_PSO J318.5-22    0
Destination_TRAPPIST-1e      0
dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py", line 420, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_p

(4277,)
Mission Successful
